#### Friday, April 19, 2024

Running locally with "lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF"

I ONLY ran the "chain.invoke" cells for local models without the "if useOpenAI:" statement at the beginning.

#### Wednesday, April 17, 2024

mamba activate langchain3

Dammit! Wrapping the code in the 'if useOpenAI:' statement supresses the output!! So yeah, stop doing that!

In [1]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="My name is System, it's true,\nI'm a language model, here to assist you!\nI'm AI-powered, with skills so fine,\nTo help with questions and answers that shine!", role='assistant', function_call=None, tool_calls=None)


In [2]:
useOpenAI = False

In [3]:
import os
from langchain_openai import ChatOpenAI

if useOpenAI:
    model = ChatOpenAI(model="gpt-3.5-turbo", api_key=os.environ['OPENAI_API_KEY_'],  temperature=0)
else:
    model = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio", temperature=0)

print(useOpenAI)

False


# Output Parsing

This notebook covers functionality related to output parsing. For more information, see:

- [List of all Output Parsers](https://python.langchain.com/docs/modules/model_io/output_parsers/)

- [OpenAI Function Calling](https://python.langchain.com/docs/modules/model_io/output_parsers/types/openai_functions)

## Converting Messages

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [5]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
# model = ChatOpenAI()

In [6]:
chain = prompt | model

In [30]:
# nothing gets returned here ... 
if useOpenAI:
    chain.invoke({"topic": "bears"})

In [45]:
# nothing gets returned here ... 
# Dammit! the if statement is supressing the output!!
# if useOpenAI:
chain.invoke({"topic": "bears"})

AIMessage(content="Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!", response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 13, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-40670f67-a53a-42a1-8d0e-6df3e0de91b9-0')

In [13]:
# I run this multiple times, but I see nothing returned, even though I see stuff happening in LMStudio.
if not useOpenAI:
    chain.invoke({"topic": "bears"})

In [7]:
# if not useOpenAI:
chain.invoke({"topic": "bears"})

AIMessage(content='Why did the bear go to the doctor?\n\nBecause it had a grizzly cough!', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 17, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bb76ab81-5a35-4de5-ab63-3b93e1d1a9ea-0')

In [8]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = prompt | model | parser

In [32]:
if useOpenAI:
    chain.invoke({"topic": "bears"})

In [47]:
# if useOpenAI:
chain.invoke({"topic": "bears"})

"Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!"

In [16]:
# running this multiple times I do not see any results being shown ... but the LMStudio logs does show action ...
if not useOpenAI:
    chain.invoke({"topic": "bears"})

In [9]:
# if not useOpenAI:
chain.invoke({"topic": "bears"})

'Why did the bear go to the doctor?\n\nBecause it had a grizzly cough!'

## OpenAI Function Calling

In [10]:
from langchain.utils.openai_functions import (
    convert_pydantic_to_openai_function,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

In [11]:
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


openai_functions = [convert_pydantic_to_openai_function(Joke)]

/home/rob/miniforge3/envs/langchain3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [12]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [13]:
parser = JsonOutputFunctionsParser()

In [14]:
chain = prompt | model.bind(functions=openai_functions) | parser

In [38]:
if useOpenAI:
    chain.invoke({"topic": "bears"})

In [53]:
# if useOpenAI:
chain.invoke({"topic": "bears"})

{'setup': 'Why do bears have hairy coats?',
 'punchline': 'Because they look grizzly without them!'}

In [22]:
# This fails on use of the work ...
# KeyError: 'function_call'
if not useOpenAI:
    chain.invoke({"topic": "bears"})

OutputParserException: Could not parse function call: 'function_call'

In [15]:
#if not useOpenAI:
chain.invoke({"topic": "bears"})

OutputParserException: Could not parse function call: 'function_call'

## Streaming

In [39]:
# For this we do get stuff coming back ... 
if useOpenAI:
    for s in chain.stream({"topic": "bars"}):
        print(s)

{}
{'setup': ''}
{'setup': 'Why'}
{'setup': 'Why did'}
{'setup': 'Why did the'}
{'setup': 'Why did the math'}
{'setup': 'Why did the math book'}
{'setup': 'Why did the math book look'}
{'setup': 'Why did the math book look sad'}
{'setup': 'Why did the math book look sad at'}
{'setup': 'Why did the math book look sad at the'}
{'setup': 'Why did the math book look sad at the bar'}
{'setup': 'Why did the math book look sad at the bar?'}
{'setup': 'Why did the math book look sad at the bar?', 'punchline': ''}
{'setup': 'Why did the math book look sad at the bar?', 'punchline': 'Because'}
{'setup': 'Why did the math book look sad at the bar?', 'punchline': 'Because it'}
{'setup': 'Why did the math book look sad at the bar?', 'punchline': 'Because it had'}
{'setup': 'Why did the math book look sad at the bar?', 'punchline': 'Because it had too'}
{'setup': 'Why did the math book look sad at the bar?', 'punchline': 'Because it had too many'}
{'setup': 'Why did the math book look sad at the bar

In [54]:
#if useOpenAI:
for s in chain.stream({"topic": "bars"}):
    print(s)

{}
{'setup': ''}
{'setup': 'Why'}
{'setup': 'Why did'}
{'setup': 'Why did the'}
{'setup': 'Why did the math'}
{'setup': 'Why did the math book'}
{'setup': 'Why did the math book look'}
{'setup': 'Why did the math book look sad'}
{'setup': 'Why did the math book look sad at'}
{'setup': 'Why did the math book look sad at the'}
{'setup': 'Why did the math book look sad at the bar'}
{'setup': 'Why did the math book look sad at the bar?'}
{'setup': 'Why did the math book look sad at the bar?', 'punchline': ''}
{'setup': 'Why did the math book look sad at the bar?', 'punchline': 'Because'}
{'setup': 'Why did the math book look sad at the bar?', 'punchline': 'Because it'}
{'setup': 'Why did the math book look sad at the bar?', 'punchline': 'Because it had'}
{'setup': 'Why did the math book look sad at the bar?', 'punchline': 'Because it had too'}
{'setup': 'Why did the math book look sad at the bar?', 'punchline': 'Because it had too many'}
{'setup': 'Why did the math book look sad at the bar

In [24]:
# This returns nothing because of that KeyError: 'function_call' error ...
if not useOpenAI:
    for s in chain.stream({"topic": "bars"}):
        print(s)

In [16]:
#if not useOpenAI:
# Nothing gets returned from the stream AND no errors shown!
for s in chain.stream({"topic": "bars"}):
    print(s)